# Pistachio simulation

<!-- Index of contents -->
* [Define the problem](#Define-the-problem)
* [Simulation agents](#Simulation-agents)
* [Stress types](#Stress-types)
  * [Chill hours stress](#Chill-hours-stress)
  * [Heat units stress](#Heat-units-stress)
  * [Temperature stress](#Temperature-stress)
  * [Hydric stress](#Hydric-stress)
* [Simulation parameters](#Simulation-parameters)

## Define the problem

......

## Simulation agents

In [1187]:
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Literal, Optional
from datetime import date
import numpy as np
from typing import Tuple

### Weather agent

In [1188]:
# Meteorological Class
class Meteorological(BaseModel):
    timestamp: date = Field(..., description="Date of the meteorological data")
    t_med: float = Field(..., ge=-30, le=50, description="Current temperature [-30°C to 50°C]")
    t_min: float = Field(..., ge=-30, le=50, description="Minimum temperature [-30°C to 50°C]")
    t_max: float = Field(..., ge=-30, le=50, description="Maximum temperature [-30°C to 50°C]")
    rh_med: float = Field(..., ge=0, le=100, description="Current relative humidity [0-100%]")
    rh_min: float = Field(..., ge=0, le=100, description="Minimum relative humidity [0-100%]")
    rh_max: float = Field(..., ge=0, le=100, description="Maximum relative humidity [0-100%]")
    wind: float = Field(..., ge=0, description="Current wind speed [0-.. m/s]")
    pp: float = Field(..., ge=0, description="Precipitation level [0-.. mm]")

# Weather Class
class Weather(BaseModel):
    # Input variables
    location: str = Field(..., description="Geographic location, e.g., Valladolid")
    init_date: date = Field(..., description="Initial date")
    end_date: date = Field(..., description="End date")
    type: Literal["normal", "extreme"] = Field(..., description="Type of weather [normal, extreme]")
    # Local variables
    meteorological: List[Meteorological] = Field(default=[], description="List of meteorological data")

    def transform_meteorological_data(self):
        match self.type:
            case "extreme":
                # NOTE: This is a dummy example, in a real case
                for meteorological in self.meteorological:
                    meteorological.temperature += np.random.choice([-5, 5])
                    meteorological.relative_humidity += np.random.choice([-10, 10])
                    meteorological.wind += np.random.choice([-10, 10])
                    meteorological.precipitation += np.random.choice([-10, 10])

    @property
    def idema(self):
        # TODO:
        return "2044B"

    def get_weather_data(self):
        # TODO: Call the API and get the weather data
        import dotenv
        import os
        import requests
        dotenv.load_dotenv()
        weather_url = os.getenv("AGROSLAB_API_URL")
        AUTH_TOKEN = os.getenv("AGROSLAB_AUTH_TOKEN")

        body = {
            "operation": "aemetclimatologiadiaria",
            "initdate": self.init_date.strftime("%d-%m-%Y"),
            "enddate": self.end_date.strftime("%d-%m-%Y"),
            "idema": self.idema
        }
        headers = {
            'Authorization': AUTH_TOKEN,
        }
        print(body)
        data = requests.post(weather_url, json=body, headers=headers)
        if data.status_code != 200:
            raise ValueError(data.json())
        historical_data_raw = data.json()
        # Process
        # tmed, velmedia, tmin, tmax, prec, date, hrMedia, hrMax, hrMin
        historical_data = []
        for data in historical_data_raw:
            meteorological = Meteorological(
                timestamp=date.fromisoformat(data["fecha"]),
                t_med=float(data.get("tmed", "0").replace(",", ".")),
                t_min=float(data.get("tmin", "0").replace(",", ".")),
                t_max=float(data.get("tmax", "0").replace(",", ".")),
                rh_med=float(data.get("hrMedia", "40").replace(",", ".")),
                rh_min=float(data.get("hrMin", "0").replace(",", ".")),
                rh_max=float(data.get("hrMax", "0").replace(",", ".")),
                wind=float(data.get("velmedia", "0").replace(",", ".")),
                pp=float(data.get("prec", "0").replace(",", "."))
            )
            historical_data.append(meteorological)
        # value
        self.meteorological = historical_data
        return historical_data

### Disease agents

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

| Disease          | Temperature       | Humidity     | Severity   | Treatment difficulty | Period               |
|------------------|-------------------|--------------|------------|-----------------------|----------------------|
| Verticillium     | $20 - 30^{\circ}C$ | Better high  | Very high  | Very high            | June - September     |
| Botryosphaeria   | $27 - 33^{\circ}C$ | Better high  | High       | Medium               | June - September     |
| Alternaria       | $27 - 35^{\circ}C$ | Better high  | Medium     | Medium               | mid July - September |
| Septoria         | $18 - 26^{\circ}C$ | Better high  | High       | Medium               | May - September      |
| Aflatoxins       | $27 - 40^{\circ}C$ | Better high  | Very High  | High                 | Mid August - Harvest |


In [1189]:
from enum import Enum

class DiseaseType(str, Enum):
    Verticillium = "Verticillium"
    Botryosphaeria = "Botryosphaeria"
    Alternaria = "Alternaria"
    Septoria = "Septoria"
    Aflatoxin = "Aflatoxin"

class Disease(BaseModel):
    type: DiseaseType = Field(..., description="Type of disease, e.g., Verticillium, Alternaria")
    RH_disease_min: float = Field(default=65, ge=0, le=100, description="Minimum relative humidity for disease")
    T_disease_min: float = Field(..., ge=0, le=50, description="Minimum temperature for disease")
    T_disease_max: float = Field(..., ge=0, le=50, description="Maximum temperature for disease")
    severity: float = Field(..., ge=0, le=1.0, description="Severity of the disease [low=0.0, high=1.0]")
    treatment_difficulty: float = Field(..., ge=0, le=1.0, description="Difficulty to treat [low=0.0, high=1.0]")
    period_init: int = Field(..., ge=0, le=365, description="Month of the year when the disease can appear")
    period_end: int = Field(..., ge=0, le=365, description="Month of the year when the disease disappears")

    # Allow arbitrary types (ndarray in this case)
    class Config:
        arbitrary_types_allowed = True

    @field_validator('period_end')
    def check_period(cls, period_end, values):
        if period_end < values['period_init']:
            raise ValueError('period_end must be greater than period_init')
        return period_end
        

# Verticillium Class
class Verticillium(Disease):
    type: DiseaseType = DiseaseType.Verticillium
    T_disease_min: float = 20
    T_disease_max: float = 30
    severity: float = np.random.uniform(0.8, 0.9)
    treatment_difficulty: float = np.random.uniform(0.8, 0.9)
    period_init: int = 6 # June
    period_end: int = 9 # September

# Botryosphaeria Class
class Botryosphaeria(Disease):
    type: DiseaseType = DiseaseType.Botryosphaeria
    T_disease_min: float = 27
    T_disease_max: float = 33
    severity: float = np.random.uniform(0.5, 0.7)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 6 # June
    period_end: int = 9 # September

# Alternaria Class
class Alternaria(Disease):
    type: DiseaseType = DiseaseType.Alternaria
    T_disease_min: float = 27
    T_disease_max: float = 35
    severity: float = np.random.uniform(0.3, 0.6)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 7 # July
    period_end: int = 9 # September

# Septoria Class
class Septoria(Disease):
    type: DiseaseType = DiseaseType.Septoria
    T_disease_min: float = 18
    T_disease_max: float = 26
    severity: float = np.random.uniform(0.5, 0.7)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 5 # May
    period_end: int = 9 # September

# Aflatoxin Class
class Aflatoxin(Disease):
    type: DiseaseType = DiseaseType.Aflatoxin
    T_disease_min: float = 25
    T_disease_max: float = 35
    severity: float = np.random.uniform(0.8, 0.95)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 8 # August
    period_end: int = 10 # October

### Pest agents

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

| Pest               | Temperature       | Humidity     | Severity | Treatment difficulty | Period             |
|--------------------|-------------------|--------------|----------|-----------------------|--------------------|
| Green stink bug    | $20 - 35^{\circ}C$ | Better high  | Medium   | High                 | August - October   |
| Pistachio psylla   | $20 - 35^{\circ}C$ | Better high  | High     | High                 | April - October    |
| Leaf beetle        | $20 - 35^{\circ}C$ | Better high  | Low      | Medium               | May                |

In [1190]:
from enum import Enum

class PestType(str, Enum):
    green_stink_bug = "Green stink bug"
    pistachio_psylla = "Pistachio psylla"
    leaf_beetle = "Leaf beetle"

class Pest(BaseModel):
    type: PestType = Field(..., description="Type of pest, e.g., Green stink bug, Leaf beetle")
    RH_pest_min: float = Field(default=65, ge=0, le=100, description="Minimum relative humidity for pest activity")
    T_pest_min: float = Field(..., ge=0, le=50, description="Minimum temperature for pest activity")
    T_pest_max: float = Field(..., ge=0, le=50, description="Maximum temperature for pest activity")
    severity: float = Field(..., ge=0, le=1.0, description="Severity of pest [low=0.0, high=1.0]")
    treatment_difficulty: float = Field(..., ge=0, le=1.0, description="Difficulty to treat [low=0.0, high=1.0]")
    period_init: int = Field(..., ge=0, le=365, description="Month of the year when the pest can appear")
    period_end: int = Field(..., ge=0, le=365, description="Month of the year when the pest disappears")

    @field_validator('period_end')
    def check_period(cls, period_end, values):
        if 'period_init' in values and period_end < values['period_init']:
            raise ValueError("period_end must be after period_init")
        return period_end
    
# Green stink bug Class
class Green_stink_bug(Pest):
    type: PestType = PestType.green_stink_bug
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.3, 0.6)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 8 # June
    period_end: int = 10 # October

# Pistachio psylla Class
class Pistachio_psylla(Pest):
    type: PestType = PestType.pistachio_psylla
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.6, 0.8)
    treatment_difficulty: float = np.random.uniform(0.6, 0.8)
    period_init: int = 4 # April
    period_end: int = 10 # October

# Leaf beetle Class
class Leaf_beetle(Pest):
    type: PestType = PestType.leaf_beetle
    T_pest_min: float = 20
    T_pest_max: float = 35
    severity: float = np.random.uniform(0.1, 0.3)
    treatment_difficulty: float = np.random.uniform(0.3, 0.6)
    period_init: int = 5 # May
    period_end: int = 5 # May

### Farm agent

In [1191]:
class Farm(BaseModel):
    location: str = Field(..., description="Geographic location, e.g., Valladolid")
    irrigation: Literal["rainfed", "irrigated"] = Field(..., description="Type of irrigation [drip, sprinkler]")

class Soil(BaseModel):
    drainage: float = Field(..., ge=0.0, le=1.0, description="Drainage quality [low=0.0, high=1.0]")

### Crop agent

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

#### Rootstock

| **Characteristic**                 | **Rootstocks (in order of preference)**                                                                                  |
|------------------------------------|--------------------------------------------------------------------------------------------------------------------------|
| Cold Resistance                    | *P. Cornicabra (or P. Terebinthus)*, *P. Atlantica*                                                                       |
| Resistance to Verticillium         | *P. Integerrima (PGI)*, UCB-1                                                                                            |
| Salinity Resistance                | *P. Atlantica*, *P. Cornicabra*                                                                                          |
| Good Productivity in Poor Soils    | *P. Cornicabra*, *P. Vera*                                                                                               |
| High Yield                         | UCB-1, *P. Integerrima*, *P. Atlantica*, *P. Cornicabra*                                                                 |
| High Vigor (Trunk Diameter)        | *P. Atlantica*, *P. Cornicabra*, *P. Integerrima*, *P. Vera*                                                             |
| Recommended for Dryland Farming    | *P. Cornicabra*                                                                                                          |
| Recommended for Irrigated Farming  | *P. Atlantica*, UCB-1                                                                                                    |


In [ ]:
from enum import Enum
from typing import Union

class RootstockType(str, Enum):
    P_Cornicabra = "P. Cornicabra"
    P_Atlantica = "P. Atlantica"
    UCB_1 = "UCB-1"
    P_Vera = "P. Vera"
    P_Integerrima = "P. Integerrima"

# class Rootstock_P_Cornicabra(BaseModel):
#     type: RootstockType = RootstockType.P_Cornicabra
#     cold_resistance: float = Field(0.15, ge=0, le=1.0, description="Cold resistance level [low=0.0, high=1.0]")

# class Rootstock_P_Atlantica(BaseModel):
#     type: RootstockType = RootstockType.P_Atlantica
#     vigour_increase: float = Field(0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
#     cold_resistance: float = Field(0.15, ge=0, le=1.0, description="Cold resistance level [low=0.0, high=1.0]")

# class Rootstock_UCB_1(BaseModel):
#     type: RootstockType = RootstockType.UCB_1
#     resistance_to_verticillium: float = Field(0.4, ge=0, le=1.0, description="Resistance to Verticillium level [low=0.0, high=1.0]")

# class Rootstock_P_Vera(BaseModel):
#     type: RootstockType = RootstockType.P_Vera
#     vigour_increase: float = Field(0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")

# class Rootstock_P_Integerrima(BaseModel):
#     type: RootstockType = RootstockType.P_Integerrima
#     vigour_increase: float = Field(0.15, ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
#     resistance_to_verticillium: float = Field(0.3, ge=0, le=1.0, description="Resistance to Verticillium level [low=0.0, high=1.0]")


# Variety Class
class Variety(BaseModel):
    HU_optimal: float = Field(..., ge=0, le=5000, description="Optimal heat units for growth")
    T_base_HU: float = Field(..., ge=0, le=50, description="Base temperature for heat units")
    CH_optimal: float = Field(..., ge=0, le=5000, description="Optimal chill hours")
    T_base_CH: float = Field(..., ge=0, le=50, description="Base temperature for chill hours")
    T_min: float = Field(..., ge=-50, le=50, description="Min temperature that variety holds")
    T_max: float = Field(..., ge=-50, le=50, description="Max temperature that variety holds")
    alternate_bearing: float = Field(..., ge=0, le=1.0, description="Level of alternate bearing [low=0.0, high=1.0]")
    vigour: float = Field(..., ge=0, le=1.0, description="Vigor level [low=0.0, high=1.0]")
    RH_min_pollination: float = Field(..., ge=0, le=100, description="Minimum relative humidity for pollination")
    RH_max_pollination: float = Field(..., ge=0, le=100, description="Maximum relative humidity for pollination")
    PP_min_year: float = Field(400, ge=0, description="Minimum precipitation per year")
    PP_optimal_year: float = Field(600, ge=0, description="Optimal precipitation per year")
    PP_max_may: float = Field(45, ge=0, description="Maximum precipitation in May")
    PP_max_april: float = Field(50, ge=0, description="Maximum precipitation in April")
    pp_max_sept: float = Field(30, ge=0, description="Maximum precipitation in September")
    tmed_list: List[float] = Field(..., description="List of daily average temperatures")
    tmin_med: float = Field(..., description="List of monthly minimum temperatures")
    tmax_med: float = Field(..., description="List of monthly maximum temperatures")
    #diseases: List[str] = Field(default_factory=list, description="List of diseases affecting the variety")
    #pests: List[str] = Field([*PestType] , description="List of pests affecting the variety")
    rootstock: RootstockType = Field(RootstockType.P_Vera, description="Type of rootstock")
    y_base_rainfed: float = Field(8, ge=0, description="Base yield for rainfed conditions")
    y_base_irrigated: float = Field(15, ge=0, description="Base yield for irrigated conditions")
    max_hours_under_extreme_temperatures: float = Field(0, ge=0, description="Maximum hours under extreme temperatures")
    CH: int = 0
    HU: float = 0

    def calculate_chill_hours(self, t_max_med_ch:float, t_min_med_ch:float):
        ch_map_weinberg = {
            
        T_ch = 0.5 * (t_max_med_ch + t_min_med_ch)


# Crop Class
class Crop(BaseModel):
    age: int = Field(10, ge=10, le=100, description="Initial age of the crop [10-100 years]")
    variety: Variety

## Stress types

### Base stress

In biological systems, these processes are usually non-linear \cite{natalia2012}. For instance, when a disease spreads, it begins gradually and takes time to build up. However, as time passes, it spreads more rapidly and through more areas, making it an almost exponential process. Based on this foundation, we will create the following functions, each with the following common elements:

<ul>
<li>
    <strong>Trigger:</strong> <code>&phi;_{trigger}</code> is a function that is 1 when optimal conditions for the stress are accomplished and 0 when not.
</li>
    <li>
        <strong>Evolution function:</strong> It describes how the base stress evolves, due to direct causes described in table <a href="#tab:stress_causes">Table 1</a>. This ranges between 0 and 1 and depends on the relevant parameter. For example, in the case of cold-hour stress, it depends on the number of cold hours. Inside, there is a growth constant (<code>&alpha;_{stress}</code>): This regulates the speed at which stress develops. The range is from -10 to 10. -10 means exponential, 0 means linear and 10 means logarithmic (<code>"growth_rate"</code> in table <a href="#tab:initial-params-desc">Table 2</a>).
    </li>
    <li>
        <strong>Existing stress:</strong> Previous stress in the current year.
    </li>
    <li>
        <strong>Random component:</strong> This corresponds to a statistical distribution to generate randomness (<code>"random_component"</code> in table <a href="#tab:initial-params-desc">Table 2</a>).
    </li>
    <li>
        <strong>Other stressors that accentuate this stress:</strong> Certain types of stress, when they reach a certain level, can trigger or exacerbate other types of stress. For example, poorly pruned branches can cause wounds (mechanical stress), which in turn make the plant more susceptible to pests (pest stress). <code>&phi;_{i}</code> is the trigger function of the stressor and <code>&lambda;_{i}</code> is the importance given to that stressor.
    </li>
</ul>


$$
S = \phi_{trigger} \cdot ( S_0 + 
\frac{1 - e^{-\alpha_{s} |r_{s}|}}{1 - e^{-\alpha_{s}}} + 
\sum_{i=1}^{n_{s}} \lambda_{osi} \cdot \phi_{osi} \cdot S_{osi} + \epsilon_{s})
$$

$\alpha_{s} \in [-10, 10]$ and $r_{s} \in [0, 1]$

$\lambda_{osi} \in [0, 0.3]$, $\phi_{osi} = (0,1)$ and $\sum_{i=1}^{n_{stress}} \lambda_{osi} \cdot \phi_{osi} \cdot S_{osi} \leq$ max_other_stressors_value $\leq 1$

$
\phi_{i} = \begin{cases}
    1 & S_i > S_{threshold\_i} \\
    0 & S_i \le S_{threshold\_i}
\end{cases}
$

In [1193]:
class Stress(BaseModel):
    stress: float = 0.0
    weight: float = Field(1, ge=0, le=1.0, description="Stress weight [0.0-1.0]")
    ## Exponential function params
    growth_rate: float = Field(..., ge=-10, le=10, description="Growth rate [-10 to 10]")
    ## Other stressors params
    other_stressors_weights: List[float] = Field([], description="Weights of stressors [0.0-1.0]")
    other_stressors_triggers: List[float] = Field([], description="Initial causal stress weights [0.0-1.0]")
    other_stressors: List[float] = Field([], description="Other stressors [0.0-1.0]")
    ## Random component
    random_component: float = Field(..., description="Random component")

    @model_validator(mode="after")
    def check_stress_weights(self):
        if len(self.other_stressors_weights) != len(self.other_stressors_triggers):
            raise ValueError("Weights and triggers must have the same length")
        if len(self.other_stressors_weights) != len(self.other_stressors):
            raise ValueError("Weights and stressors must have the same length")
        return self
    
    @property
    def _trigger(self):
        print("Implement this method in corresponding stress class...")
        raise NotImplementedError

    @property
    def _magnitude_of_stress(self):
        print("Implement this method in corresponding stress class...")
        raise NotImplementedError

    @property
    def _evolution_function(self):
        magnitude = self._magnitude_of_stress
        return (1 - np.exp(-self.growth_rate) * np.abs(magnitude)) / (1 - np.exp(-self.growth_rate))

    @property
    def _other_stressors(self):
        trigger_function = [1 if stressor > causal_stress else 0 for stressor, causal_stress in zip(self.other_stressors, self.other_stressors_triggers)]
        return np.sum(np.multiply(self.other_stressors_weights, trigger_function))

    def calculate_stress(self):
        if self._trigger:
            self.stress += self._evolution_function \
                + self._other_stressors \
                + self.random_component
        return self.stress


### Chill hours stress

$$
    S_{chill\_hours}(CH) = \begin{cases}
    \frac{1 - e^{-\alpha_{cold} |r_{cold}|}}{1 - e^{-\alpha_{cold}}} + \epsilon & CH < CH_{optimal}\\
    0 & CH \ge CH_{optimal}
\end{cases}
$$

$CH_{optimal} =$ are the optimal chill hours for the corresponding pistachio variety.

$r_{cold} = 0 \ge \frac{CH_{optimal} - CH}{CH_{optimal}} > 1$

$\epsilon \sim N(\mu, \sigma)$

In [1194]:
class ChillHoursStress(Stress):
    # Stress input params
    CH: float = Field(..., ge=0, le=5000, description="Chill hours")
    # Stress specific local params
    CH_optimal: float = Field(..., ge=0, le=5000, description="Optimal chill hours")

    @property
    def _trigger(self):
        return self.CH < self.CH_optimal

    @property
    def _magnitude_of_stress(self):
        return min(self.CH / self.CH_optimal, 1)

### Heat units stress

$$
    S_{chill\_hours}(CH) = \begin{cases}
    \frac{1 - e^{-\alpha_{cold} |r_{cold}|}}{1 - e^{-\alpha_{cold}}} + \epsilon & CH < CH_{optimal}\\
    0 & CH \ge CH_{optimal}
\end{cases}
$$

$HU_{optimal} =$ are the optimal heat units for the corresponding pistachio variety.

$r_{heat} = 0 \ge \frac{HU_{optimal} - HU}{HU_{optimal}} > 1$

$\epsilon \sim N(\mu, \sigma)$

In [1195]:
class HeatUnitsStress(Stress):
    # Stress input params
    HU: float = Field(..., ge=0, le=5000, description="Heat units")
    # Stress specific local params
    HU_optimal: float = Field(..., ge=0, le=5000, description="Optimal heat units")

    @property
    def _trigger(self):
        return self.HU < self.HU_optimal

    @property
    def _magnitude_of_stress(self):
        return min(self.HU / self.HU_optimal, 1)

### Temperature stress

$$
S_{temp}(H_{T\_ext}) = \begin{cases}
    S_{temp\_0} + \frac{1 - e^{-\alpha_{temp} |r_{temp}|}}{1 - e^{-\alpha_{temp}}} + \epsilon & H_{T\_ext} > 0\\
    0 & H_{T\_ext} = 0
\end{cases}
$$

$H_{T\_ext} = hours(T \ge T_{max} \vee T_{min} \ge T)$

$r = 0 \ge \frac{H_{T\_ext}}{H_{T\_ext\_max}} \ge 1$

$H_{T\_ext\_max} =$ hours of extreme temperatures between $T_{min}$ and $T_{max}$.

$\epsilon \sim N(\mu, \sigma)$

In [1196]:
class TemperatureStress(Stress):
    # Stress input params
    hours_under_extreme_temperature: int = Field(..., ge=0, description="Hours under extreme temperature")
    # Stress specific local params
    max_hours_under_extreme_temperature: int = Field(730, ge=0, le=24, description="Max hours under extreme temperature")
    
    @property
    def _trigger(self):
        return self.hours_under_extreme_temperature > 0

    @property
    def _magnitude_of_stress(self):
        return min(self.hours_under_extreme_temperature / self.max_hours_under_extreme_temperature, 1)

### Hydric stress

$$
S_{hydric}(PP) = \begin{cases}
    S_{hydric\_0} + \frac{1 - e^{-\alpha_{hydric} |r_{hydric}|}}{1 - e^{-\alpha_{hydric}}} + \epsilon & PP < PP_{min} \vee PP > PP_{max}\\
    S_{hydric\_0} & PP_{min} \ge PP \ge PP_{max}
\end{cases}
$$

$r_{hydric} = 0 \ge \frac{PP_{optimal} - PP}{PP_{optimal} - PP_{min}} \ge 1$

In [1197]:
class HydricStress(Stress):
    # Stress input params
    PP: float = Field(..., ge=0, le=100, description="Precipitation level")
    # Stress specific local params
    PP_min: float = Field(..., ge=0, le=100, description="Minimum precipitation level")
    PP_max: float = Field(..., ge=0, le=100, description="Maximum precipitation level")

    @property
    def _trigger(self):
        return self.PP < self.PP_min or self.PP > self.PP_max

    @property
    def _calculate_magnitude_of_stress(self):
        # TODO: PENSARR BIEEEEN!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        if self.PP < self.PP_min:
            return self.PP / self.PP_min
        else:
            return self.PP / self.PP_max

### Mechanical stress

$$
\small
S_{mechanical} = S_{mechanical\_0} + 1 - M
$$

$M \sim N(\mu, \sigma)$

In [1198]:
class MechanicalStress(Stress):
    # Stress specific local params
    farming_activity: float = Field(..., description="Farming activity")

    @property
    def _trigger(self):
        return True

    @property
    def _evolution_function(self):
        return self.farming_activity

### Nutritional stress

$$
S_{nutri} = \begin{cases}
 S_{nutri\_0} + 1 - N + \sum_{i=1}^{n\_stress} \lambda_{i} \cdot \phi_{i} \cdot S_{i} & \text{fertilisation}\\
 S_{nutri\_0} & \text{no fertilisation}
\end{cases}
$$

$N \sim N(\mu, \sigma)$ represents the success of fertilisation (fert) activity between 0 and 1.

$n\_estres =$ \{hydric\}

$
\phi_{i} = \begin{cases} 
    1 & S\_hydric > 0.5 \\
    0 & S\_hydric \le 0.5 \\
\end{cases}
$

$epsilon \sim N(\mu, \sigma)$

In [1199]:
class NutritionalStress(Stress):
    # Stress specific local params
    fertilisation: float = Field(..., description="Fertilisation activity")

    @property
    def _trigger(self):
        return True

    @property
    def _evolution_function(self):
        return self.fertilisation

### Pest and disease stress

$$
S_{pest}(H_{RH\_pest}, H_{T\_pest}) = \begin{cases}
    S_{pest\_0} + \frac{1 - e^{-\alpha_{pest} |r_{pest}|}}{1 - e^{-\alpha_{pest}}} + \sum_{i=1}^{n_{stress}} \lambda_{i} \cdot \phi_{i} \cdot S_{i} + \epsilon & H_{RH\_pest} > 0 \wedge H_{T\_pest} > 0 \\
    S_{pest\_0} & H_{RH\_pest} = 0 \vee H_{T\_pest} = 0
\end{cases}\\
$$

$H_{RH\_pest} = hours(RH \ge RH\_pest\_min)$ \& $H_{T\_pest} = hours(T_\_pest\_min \le T \le T_\_pest\_max)$ These are the hours of exposition to favourable relative humidity and temperature of pest.

$n_{stress} =$ \{mechanical, heat\_units\}

$\alpha_{pest}$ is the severity of the pest, that represents the growth speed of the pest.

$r_{pest} = 0 \ge \frac{H_{RH\_pest}}{H_{RH\_pest\_max}} \ge 1$

$\epsilon \sim N(\mu, \sigma)$

In [1200]:
class PestDiseaseStress(Stress):
    # Stress input params
    hours_under_optimal_temperature: int = Field(0, ge=0, description="Hours under optimal temperature in a period")
    hours_under_optimal_rh: int = Field(0, ge=0, description="Hours under optimal relative humidity in a period")
    # Stress specific local params
    max_hours_under_optimal_t: int = Field(..., ge=0, description="Max hours under optimal temperature")
    max_hours_under_optimal_rh: int = Field(..., ge=0, description="Max hours under optimal relative humidity")

    @property
    def _trigger(self):
        return self.hours_under_optimal_temperature > 0 and self.hours_under_optimal_rh > 0

    @property
    def _magnitude_of_stress(self):
        return min(self.hours_under_optimal_temperature / self.max_hours_under_optimal_t, 1) \
            + min(self.hours_under_optimal_rh / self.max_hours_under_optimal_rh, 1)

### Total stress

Equation shows the accumulated total stress that is calculated as the weighted average of each corresponding stress and the residual stress, where each $\lambda_{i}$ represents the weighted importance assigned to each one, $n_{stress} \in$ {chill hours, temperature, heat units, water, mechanical, nutritional, pest, disease}, $\lambda_{prev}$ is the weighted importance assigned to the previous residual stress and $S_{prev}$ is the previous residual stress of last productive years. $S_{total}$ must always be between 0 and 1.

$$
S_{total} = \lambda_{prev} \cdot S_{prev} + \frac{\sum_{i=1}^{n\_stress} \lambda_{ti} \cdot S_{i}}{\sum_{i=1}^{n\_stress} \lambda_{ti}}
$$

In [1201]:
class TotalStress(BaseModel):
    total_stress: float = Field(..., ge=0, le=1.0, description="Total stress level [0.0-1.0]")
    # Stress specific local params
    stressors: List[Tuple[float, float]] = Field(..., description="List of tuples where each tuple contains (stress_weight, stress_level)")
    previous_stress_weight: float = Field(..., ge=0, le=1.0, description="Previous stress weight [0.0-1.0]")
    previous_stress: float = Field(..., ge=0, le=1.0, description="Previous stress level [0.0-1.0]")

    def calculate(self):
        effective_previous_stress = self.previous_stress_weight * self.previous_stress
        weighted_sum = sum([stress_weight * stress for stress_weight, stress in self.stressors]) / sum([stress_weight for stress_weight, _ in self.stressors])
        self.total_stress =  effective_previous_stress + weighted_sum
        return self.total_stress

## Stress-based estimated yield equation

The following equation calculates the approximated yield per tree in a productive year, where γpollination
is the degree of female tree pollination, γbearing is the alternate bearing suffered -depends on
“on”-“off” cycle-, γage is the increase of yield that depends on the tree’s age, Stotal is the total
accumulated stress, Ybase is a baseline yield determined by extracted knowledge and ϵ is a
probabilistic distribution that adds randomness. Ybase depends on whether the tree is grown
under rainfed or irrigated conditions

$$
Y = \gamma_{pollination} \cdot (\gamma_{bearing} \cdot (1 - S_{total}) \cdot Y_{base} + \gamma_{age}) + \epsilon_{y}\\
$$

$\gamma_{age}$ represents that every two years production raises 1 kilograms in case of rainfed and 1.5 kilograms in case of irrigated.

$$
\small
\gamma_{age} = \begin{cases}
    1\ kilogram & \ge 10\ years\ old \wedge rainfed \\
    1.5\ kilograms & \ge 10\ years\ old \wedge irrigated
\end{cases}
$$

$\epsilon \sim N(\mu, \sigma)$\\

$\gamma_{bearing}$ represents the intensity of the alternate bearing and depends on the last year production and the pistachio variety. It should be between 50\% and 80\% less than the last year production, depending on the stress levels.

$$
\small
\gamma_{bearing} = \begin{cases}
     (1 - AB) \le \epsilon_{bearing} - 0.2 \cdot S_{total} \le (1 - AB + 0.3) & ``off'' year\\
     1 & ``on'' year
\end{cases}
$$

$AB$ is alternate bearing.

$\epsilon_{bearing} \sim U(0.3, 0.5)$\\

$\gamma_{pollination}$ represents the percentage of pollination of male trees to female ones in April. It ranges between 0 and 1.

$$
\small
\gamma_{pollination} = \begin{cases}
    1 - \frac{1 - e^{-\alpha_{pollination} |r_{pollination}|}}{1 - e^{-\alpha_{pollination}}} - \lambda_{CH} \cdot S_{CH} + \epsilon & avg(RH) > 85\% \vee PP > 50 mm\\
    1 & avg(RH) < 85\% \wedge PP \le 50 mm
\end{cases}
$$

# TODO: MEJOR MEDIA ARITMETICA PARA EL r_{pollination}?

$
r_{pollination} = 0 \ge \frac{avg(RH) - RH_{min}}{RH_{max} - RH_{min}} \cdot \frac{PP - PP_{min}}{PP_{max} - PP_{min}} > 1
$\\

$Y_{base}$ is the base production of the registered average according to knowledge. We have seen that is around 8 kilograms in rainfed regime and around 15 kilograms in irrigation regime.

$$
\small
Y_{base} = \begin{cases}
    8\ kilograms & \ge 10\ years\ old \wedge rainfed \\
    15\ kilograms & \ge 10\ years\ old \wedge irrigated
\end{cases}
$$

In [1202]:
from pydantic import BaseModel, Field
from typing import Literal
import numpy as np

class EstimatedYield(BaseModel):
    # Input parameters
    age: int = Field(..., ge=10, description="Age of the tree in years")
    irrigation: Literal["rainfed", "irrigated"] = Field(..., description="Irrigation type (rainfed or irrigated)")
    alternate_bearing: float = Field(..., ge=0.5, le=0.8, description="Intensity of alternate bearing (0.5 to 0.8)")
    total_stress: float = Field(..., ge=0, le=1, description="Total accumulated stress (0 to 1)")
    avg_rh_april: float = Field(..., ge=0, le=100, description="Average relative humidity in April (%)")
    pp_april: float = Field(..., ge=0, description="Precipitation in April (mm)")
    alpha_pollination: float = Field(..., ge=0, description="Pollination growth rate parameter")
    lambda_ch: float = Field(..., ge=0, le=1, description="Weight of chill hours stress in pollination")
    s_ch: float = Field(..., ge=0, le=1, description="Chill hours stress (0 to 1)")
    random_component: float = Field(..., description="Random component")
    y_base_rainfed: float = Field(8, description="Base yield for rainfed trees")
    y_base_irrigated: float = Field(15, description="Base yield for irrigated trees")
    max_pollination_rh: float = Field(85, description="Maximum relative humidity for pollination")
    max_pollination_pp: float = Field(50, description="Maximum precipitation for pollination")

    # Allow arbitrary types (ndarray in this case)
    class Config:
        arbitrary_types_allowed = True

    @property
    def is_on_year(self) -> bool:
        """Check if the tree is "on" year. In this case, if the age is even."""
        return self.age % 2 == 0

    # Computed properties
    @property
    def _gamma_age(self) -> float:
        """Compute gamma_age based on the age and irrigation type."""
        if self.age >= 10:
            return 1.5 if self.irrigation == "irrigated" else 1.0
        return 0.0

    @property
    def _gamma_bearing(self) -> float:
        # TODO: añadir alternate bearing
        """Compute gamma_bearing based on alternate bearing and stress levels."""
        epsilon_bearing = np.random.uniform(0.3, 0.5)
        if not self.is_on_year:
            return max(0.2, min(epsilon_bearing - 0.2 * self.total_stress, 0.5))
        return 1.0

    @property
    def _gamma_pollination(self) -> float:
        # TODO: QUIZAS DERIVAR DE LA CLASE Stress PORQUE LA FORMULA ES LA MISMA
        """Compute gamma_pollination based on humidity, precipitation, and stress."""
        # if self.avg_rh_april > self.max_pollination_rh or self.pp_april > self.max_pollination_pp:
        #     r_pollination = max(0, min(
        #         ((self.avg_rh_april - 60) / (self.max_pollination_rh - 60)) * ((self.pp_april - 10) / (self.max_pollination_pp - 10)), 1))
        #     exponential_term = (
        #         1 - np.exp(-self.alpha_pollination * abs(r_pollination))
        #     ) / (1 - np.exp(-self.alpha_pollination))
        #     return 1 - exponential_term - self.lambda_ch * self.s_ch + self.random_component
        # return 1.0
        return 1.0

    @property
    def _y_base(self) -> float:
        """Compute Y_base based on age and irrigation type."""
        if self.age >= 10:
            return self.y_base_rainfed if self.irrigation == "rainfed" else self.y_base_irrigated
        return 0.0

    def calculate_yield(self) -> float:
        """Calculate the estimated yield per tree."""
        random_component = self.random_component
        return (
            self._gamma_pollination *
            (self._gamma_bearing * (1 - self.total_stress) * self._y_base + self._gamma_age) +
            random_component
        )

## Simulation parameters

<style scoped>
table {
  font-size: 16px;
  font-family: Verdana, sans-serif;
}
</style>

In this case, we are going to be using the **Kerman variety** of pistachio, which is the most common variety in Spain. It is usually used with the rootstock **Pistacia atlantica**. The following table shows the initial parameters for the simulation.

Crop

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
| init_age | Initial age of the crop | 10 yo | - | - |
| $CH\_optimal$ | Optimal chill hours for Kerman variety | [800-1000] | 1000 | --- |
| $HU\_optimal$ | Optimal heat units for Kerman variety | [3300-3500] | 3500 | --- |
| $T\_min$ | Minimum temperature for the Kerman variety | [-30,-15] ºC | -30 ºC | --- |
| $T\_max$ | Maximum temperature for the Kerman variety | [40,50] ºC | 50 ºC | --- |
| $T\_base\_CH$ | Base temperature for chill hours calculation | < 7 ºC | < 7 ºC | - |
| $T\_base\_HU$ | Base temperature for heat units calculation | > 10 ºC | > 10 ºC | - |
| alternate_bearing | Alternate bearing | [0,1]* | cat | --- |
| RH_min_pollination | --- | [40,60] % | 50 % | --- |
| RH_max_pollination | --- | [75,90] % | 90 % | --- |
| PP_min_year | --- | [300,400] mm | 400 mm | --- |
| PP_optimal_year | --- | [500,600] mm | 600 mm | --- |
| PP_max_may | Maximum precipitation in May | [30,60] mm | 45 mm | --- |
| PP_max_april | Maximum precipitation in April | [40,60] mm | 50 mm | --- |
| PP_max_sept | Maximum precipitation in September | [20,40] mm | 30 mm | --- |
| rootstock | Rootstock | Pistacia atlantica | - | - |
| y_base_rainfed | --- | [6,10] Kg | 8 | --- |
| y_base_irrigated | --- | [12,17] Kg | 15 | --- |
UNKNOWN
| max_hours_under_extreme_temperatures (H_{T\_ext}) | [24\*7\*4,24\*7\*6] h | - | --- |

Soil
| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
| drainage | --- | [0,1]* | free | --- |

Farm

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
| irrigation | --- | ["rainfed", "irrigated"] | free | --- |


Weather

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
| location | --- | ["province"] | free | - |
| init_date | --- | "dd-mm-yyyy" | free | - |
| end_date | --- | "dd-mm-yyyy" | free | - |
| type | --- | ["normal","extreme"] | free | - |

Disease and pest agents [DiseaseType] (supuestamente no es input)

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
| type | --- | --- | - | - |
| RH_disease_min | --- | [55,70] % | cat | --- |
| severity | --- | - | cat | --- |
| treatment_difficulty | --- | [0,1]* | cat | --- |
| T_disease_min | --- | $t$ | num | --- |
| T_disease_max | --- | $t$ | num | --- |
| period_init | --- | [1,12] | num | - |
| period_end | --- | [1,12] | num | - |
UNKNOWN
| max_hours_under_optimal_rh | --- | [24,24*3] | - | --- |
| max_hours_under_optimal_t | --- | [24,24*3] | - | --- |

Stress

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
UNKNOWN
| weight $(\lambda_{ti})$ | --- | [0,1] | - | --- |
| growth_rate $(\alpha_{s})$ | --- | [-10,10] | - | --- |
| other_stressors_weights $(\lambda_{osi})$ | --- | [0,1] | - | --- |
| other_stressors_triggers $\phi_{osi}$ | --- | [0,1] | - | --- |
| random_component $(\epsilon_{s})$ | --- | [-0.25,0.25] | - | --- |

Total stress

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
UNKNOWN
| previous_stress_weight ($\lambda_{prev}$) | --- | [0,1] | - | --- |

Stress-based estimated yield

| Parameter | Description | Test value | Theorical value | Optimal value |
| --- | --- | --- | --- | --- |
UNKNOWN
| random_component $(\epsilon_{y})$ | --- | [-0.25,0.25] | --- |
| alpha_pollination | --- | [-10,10] | - | --- |


## Simulation

Pseudocode:

```
function simulation (init_date, end_date, input_params):
  for year in range(init_date, end_date):
    weather_agent, crop, farm, soil, diseases, pests <- reset(input_params, year)

    # Anual stress
    stress[ch_stress] <- chill_hours_stress(crop.CH)
    stress[heat_units_stress] <- heat_units_stress(crop.HU)
    stress[temp_stress] <- temperature_stress(weather.meteorologic.t_med)
    stress[mechanical_stress] <- mechanical_stress(farm.fieldwork)
    stress[nutritional_stress] <- nutritional_stress(farm.fertilisation)
    
    # Temporal restricted stress
    for stress in list(diseases, pests):
      weather_in_outbreak_period <- weather_agent.get_weather(stress.period_init, stress.period_end)
      stress[pest_stress_{pest}] <- pest_stress(weather_in_outbreak_period.RH, weather_in_outbreak_period.T)
      stress[disease_stress_{disease}] <- disease_stress(weather_in_outbreak_period.RH, weather_in_outbreak_period.T)
    
    for weather_month in weather_agent.get_weather(init_date, end_date):
      stress[hydric] <- hydric_stress + hydric_stress(weather_month.PP)
        
    total_stress <- prev_stress + weighted_sum(stress[...n])
    y <- estimated_yield(total_stress, ...otger_args)
    
    # Save synthetic data
    save_data(y, total_stress, stress[...n], ...agents)

```

In [ ]:
params = {
    "init_date": date(2022, 4, 1),
    "end_date": date(2022, 10, 1),
    "age": 10,
    "location": "Valladolid",
    "weather_type": "normal",
    "irrigation": "rainfed",
    "drainage": "high",
    "alternate_bearing": "high",
    "vigour": "medium",
    "stress": {
        "max_other_stressors_value": np.random.uniform(0.5, 1),
        "chill_hours": {
          "weight": 1,
          "other_stressors": []
        },
        "heat_units": {
          "weight": 1,
          "other_stressors": []
        },
        "temperature": {
          "weight": 1,
          "other_stressors": []
        },
        "hydric": {
          "weight": 1,
          "other_stressors": []
        },
        "mechanical": {
          "weight": 1,
          "other_stressors": []
        },
        "nutritional": {
          "weight": 1,
          "other_stressors": [
            {
              "name": "hydric",
              "weight": np.random.uniform(0, 1),
              "trigger": np.random.uniform(0, 1)
            }
          ]
        },
        "pest": {
          "other_stressors": [{
            "name": "heat_units",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          },{
            "name": "mecanical",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          }]
        },
        "disease": {
          "other_stressors": [{
            "name": "heat_units",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          },{
            "name": "pest",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          },{
            "name": "mechanical",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          },
          {
            "name": "nutritional",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          },
          {
            "name": "hydric",
            "weight": np.random.uniform(0, 1),
            "trigger": np.random.uniform(0, 1)
          }]
        }
    },
    "pests": ["Green stink bug", "Pistachio psylla", "Leaf beetle"],
    "diseases": ["Verticillium", "Botryosphaeria", "Alternaria", "Septoria", "Aflatoxin"],
    "rootstock": "P. Vera",
}

# Lo que provoca
# irrigation -> 
#   irrigated -> weight_hydric_stress = [0, 0.3]
# treatment_difficulty -> 
#   very_high -> weight_pest_disease_stress = [0.9, 1]
#   high -> weight_pest_disease_stress = [0.6, 0.9]
#   medium -> weight_pest_disease_stress = [0.4, 0.6]
#   low -> weight_pest_disease_stress = [0.2, 0.4]
# drainage ->
#   low -> weight_hydric_stress = [0.6, 1]
#   medium -> weight_hydric_stress = [0.3, 0.6]
#   high -> weight_hydric_stress = [0, 0.3]
# rootstock.P_Vera -> weight_temperature_stress--
# rootstock.P_Vera -> weight_general -
# rootstock.UCB_1 -> weight_verticillium_stress = [0, 0.2]
# rootstock.P_Cornicabra -> weight_nutritional_stress-
# rootstock.P_Cornicabra -> weight_temperature_stress--
# rootstock.P_Cornicabra -> weight_general -
# rootstock.P_Intergerrima -> weight_verticillium_stress = [0, 0.2]
# rootstock.P_Integerrima -> weight_general -
# rootstock.P_Atlantica -> weight_temperature_stress--
# rootstock.P_Atlantica -> weight_general -


def simulation(params):
  estimated_yield = 0.0
  stress = 0.0
  previous_stress = 0.0
  age = params["age"]
  weather_type = params["weather_type"]
  location = params["location"]
  init_date = params["init_date"]
  end_date = params["end_date"]
  irrigation = params["irrigation"]
  drainage = params["drainage"] or "high"

  # For every year in the simulation
  for year in range(init_date.year, end_date.year + 1):
    stress = dict()
    # Get the weather data
    weather = Weather(
      location=location,
      init_date=date(year, 10, 1),
      end_date=date(year+1, 3, 1),
      type=weather_type
    )
    weather.transform_meteorological_data()
    weather.get_weather_data()

    t_min_med_hu =  np.mean([fila.t_min for fila in weather.meteorological if 4 <= fila.timestamp.month <= 10])
    t_max_med_hu =  np.mean([fila.t_max for fila in weather.meteorological if 4 <= fila.timestamp.month <= 10])
    t_max_med_ch =  np.mean([fila.t_max for fila in weather.meteorological if 4 <= fila.timestamp.month <= 10])
    t_max_med_ch =  np.mean([fila.t_max for fila in weather.meteorological if 4 <= fila.timestamp.month <= 10])

    # Get the crop data
    crop = Crop(
      age=age,
      variety=Variety(
        HU_optimal=np.random.uniform(3300, 3500),
        T_base_HU=10,
        CH_optimal=np.random.uniform(1000, 1200),
        T_base_CH=7,
        T_min=np.random.uniform(-30, -15),
        T_max=np.random.uniform(40, 50),
        alternate_bearing=np.random.uniform(0.6, 0.8),
        vigour=np.random.uniform(0.4, 0.6),
        RH_min_pollination=np.random.uniform(40, 60),
        RH_max_pollination=np.random.uniform(75, 90),
        PP_min_year=np.random.uniform(300, 400),
        PP_optimal_year=np.random.uniform(500, 600),
        PP_max_may=np.random.uniform(40, 50),
        PP_max_april=np.random.uniform(40, 60),
        pp_max_sept=np.random.uniform(20, 40),
        rootstock=RootstockType.P_Vera,
        y_base_rainfed=np.random.uniform(6, 10),
        y_base_irrigated=np.random.uniform(12, 17),
        max_hours_under_extreme_temperatures=np.random.uniform(24*7*4, 24*7*6),
      )
    )
    farm = Farm(
      location=location,
      irrigation="rainfed"
    )
    soil = Soil(
      drainage=np.random.uniform(0.7, 1)
    )
    pests: List[PestDiseaseStress] = [
      Green_stink_bug(),
      Pistachio_psylla(),
      Leaf_beetle()
    ]
    diseases: List[PestDiseaseStress] = [
      Verticillium(),
      Botryosphaeria(),
      Alternaria(),
      Septoria(),
      Aflatoxin()
    ]

    print(crop.variety.HU, crop.variety.CH)
    
    # Anual stress
    stress["chill_hours"] = ChillHoursStress(
      CH=crop.variety.CH,
      CH_optimal=crop.variety.CH_optimal,
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()
    stress["heat_units"] = HeatUnitsStress(
      HU=crop.variety.HU,
      HU_optimal=crop.variety.HU_optimal,
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()
    hours_under_extreme_temperature = len([tmed for tmed in weather.meteorological.t_med if tmed < crop.variety.T_min or tmed > crop.variety.T_max])
    stress["temperature"] = TemperatureStress(
      hours_under_extreme_temperature=hours_under_extreme_temperature,
      max_hours_under_extreme_temperature=crop.variety.max_hours_under_extreme_temperatures,
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()
    stress["hydric"] = HydricStress(
      PP=[meteorological.pp for meteorological in weather.meteorological],
      PP_min=crop.variety.PP_min_year,
      PP_max=crop.variety.PP_optimal_year,
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()
    stress["mechanical"] = MechanicalStress(
      farming_activity=np.random.normal(0.5, 0.1),
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()
    stress["nutritional"] = NutritionalStress(
      fertilisation=np.random.normal(0.5, 0.1),
      growth_rate=np.random.uniform(-10, 10),
      random_component=np.random.uniform(-0.25, 0.25)
    ).calculate_stress()

    # Temporal restricted stress
    for pest in pests:
      weather_in_outbreak_period = [meteorological for meteorological in weather.meteorological if pest.period_init < meteorological.date.month < pest.period_end]
      pest.hours_under_optimal_temperature = len([tmed for tmed in weather_in_outbreak_period.t_med if pest.T_pest_min < tmed < pest.T_pest_max])
      pest.hours_under_optimal_rh = len([rh for rh in weather.meteorological.rh if pest.RH_pest_min < rh])
      stress["pest"][pest.type] = pest.calculate_stress()
    for disease in diseases:
      weather_in_outbreak_period = [meteorological for meteorological in weather.meteorological if disease.period_init < meteorological.date.month < disease.period_end]
      disease.hours_under_optimal_temperature = len([tmed for tmed in weather_in_outbreak_period.t_med if disease.T_disease_min < tmed < disease.T_disease_max])
      disease.hours_under_optimal_rh = len([rh for rh in weather.meteorological.rh if disease.RH_disease_min < rh])
      stress["disease"][disease.type] = disease.calculate_stress()
    ## April hydric stress
    weather_in_april = [meteorological for meteorological in weather.meteorological if meteorological.date.month == 4]
    pp_april = np.sum([meteorological.pp for meteorological in weather_in_april])
    avg_rh_april = np.mean([meteorological.rh for meteorological in weather_in_april])
    stress["hydric"] += HydricStress(
      PP=pp_april,
      PP_min=crop.variety.PP_max_april,
      PP_max=crop.variety.PP_max_april
    ).calculate_stress()
    ## May hydric stress
    weather_in_may = [meteorological for meteorological in weather.meteorological if meteorological.date.month == 5]
    pp_may = np.sum([meteorological.pp for meteorological in weather_in_may])
    stress["hydric"] += HydricStress(
      PP=pp_may,
      PP_min=crop.variety.PP_max_may,
      PP_max=crop.variety.PP_max_may
    ).calculate_stress()
    ## September hydric stress
    weather_in_sept = [meteorological for meteorological in weather.meteorological if meteorological.date.month == 9]
    pp_sept = np.sum([meteorological.pp for meteorological in weather_in_sept])
    stress["hydric"] += HydricStress(
      PP=pp_sept,
      PP_min=crop.variety.pp_max_sept,
      PP_max=crop.variety.pp_max_sept
    ).calculate_stress()
    
    # Total stress
    stress = TotalStress(
      total_stress=0,
      stressors=[
        (stress["chill_hours"].weight, stress["chill_hours"]),
        (stress["heat_units"].weight, stress["heat_units"]),
        (stress["temperature"].weight, stress["temperature"]),
        (stress["hydric"].weight, stress["hydric"]),
        (stress["mechanical"].weight, stress["mechanical"]),
        (stress["nutritional"].weight, stress["nutritional"]),
        (stress["pest"]["Green stink bug"].weight, stress["pest"]["Green stink bug"]),
        (stress["pest"]["Pistachio psylla"].weight, stress["pest"]["Pistachio psylla"]),
        (stress["pest"]["Leaf beetle"].weight, stress["pest"]["Leaf beetle"]),
        (stress["disease"]["Verticillium"].weight, stress["disease"]["Verticillium"]),
        (stress["disease"]["Botryosphaeria"].weight, stress["disease"]["Botryosphaeria"]),
        (stress["disease"]["Alternaria"].weight, stress["disease"]["Alternaria"]),
        (stress["disease"]["Septoria"].weight, stress["disease"]["Septoria"]),
        (stress["disease"]["Aflatoxin"].weight, stress["disease"]["Aflatoxin"]),
      ],
      previous_stress_weight=np.random.uniform(0.1, 0.3),
      previous_stress=previous_stress
    ).calculate()

    # Estimated yield
    estimated_yield = EstimatedYield(
      age=age,
      irrigation=irrigation,
      alternate_bearing=crop.variety.alternate_bearing,
      total_stress=stress,
      avg_rh_april=avg_rh_april,
      pp_april=pp_april,
      alpha_pollination=np.random.uniform(-10, 10),
      lambda_ch=np.random.uniform(0, 1),
      s_ch=stress["chill_hours"],
      random_component=np.random.uniform(-0.25, 0.25),
      y_base_rainfed=crop.variety.y_base_rainfed,
      y_base_irrigated=crop.variety.y_base_irrigated,
      max_pollination_rh=crop.variety.RH_max_pollination,
      max_pollination_pp=crop.variety.PP_max_may
    ).calculate_yield()

    # Save to file
    print(f"{year}, {estimated_yield}, {stress}")
    with open(f"results.txt", "w") as f:
      f.write(f"{year}, {estimated_yield}, {stress}\n")
    
    # Update the age
    age += 1
    previous_stress = stress
    

In [1204]:
# weather = Weather(
#   location=params["location"],
#   init_date=params["init_date"],
#   end_date=params["end_date"],
#   type=params["weather_type"]
# )
# weather.transform_meteorological_data()
# weather.get_weather_data()

# crop = Crop(
#   age=params["age"],
#   variety=Variety(
#     HU_optimal=np.random.uniform(3300, 3500),
#     T_base_HU=10,
#     CH_optimal=np.random.uniform(1000, 1200),
#     T_base_CH=7,
#     T_min=np.random.uniform(-30, -15),
#     T_max=np.random.uniform(40, 50),
#     alternate_bearing=np.random.uniform(0.6, 0.8),
#     vigour=np.random.uniform(0.4, 0.6),
#     RH_min_pollination=np.random.uniform(40, 60),
#     RH_max_pollination=np.random.uniform(75, 90),
#     PP_min_year=np.random.uniform(300, 400),
#     PP_optimal_year=np.random.uniform(500, 600),
#     PP_max_may=np.random.uniform(40, 50),
#     PP_max_april=np.random.uniform(40, 60),
#     pp_max_sept=np.random.uniform(20, 40),
#     tmed_list=[meteorological.t_med for meteorological in weather.meteorological],
#     tmin_list=[meteorological.t_min for meteorological in weather.meteorological],
#     tmax_list=[meteorological.t_max for meteorological in weather.meteorological],
#     rootstock=RootstockType.P_Vera,
#     y_base_rainfed=np.random.uniform(6, 10),
#     y_base_irrigated=np.random.uniform(12, 17),
#     max_hours_under_extreme_temperatures=np.random.uniform(24*7*4, 24*7*6),
#   )
# )

# weather.meteorological

In [1205]:
simulation(params)

{'operation': 'aemetclimatologiadiaria', 'initdate': '01-10-2022', 'enddate': '01-03-2023', 'idema': '2044B'}
3240.0048387096776 1168


AttributeError: 'list' object has no attribute 't_med'

In [ ]:
# Normal distribution with mean=0 and std=1
import numpy as np
np.random.normal(0, 1)

-0.4259842234268147

### Happy path

In [ ]:
pest_stress = PestDiseaseStress(
    max_hours_under_optimal_t=100,
    max_hours_under_optimal_rh=100,
    weight=np.random.uniform(0, 1),
    stressor_weight=[0.5, 0.5],
    causal_stress_init=[0.5, 0.5],
    other_stressors=[0.5, 0.5],
    corrective_constants=[0.5, 0.5],
    random_component=0.5
)

np.random.normal(0.5, 0.1)

ValidationError: 1 validation error for PestDiseaseStress
growth_rate
  Field required [type=missing, input_value={'max_hours_under_optimal...'random_component': 0.5}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing